# **IMPORTING LIBRARIES**


In [ ]:
import torch
import torchvision
import numpy as np
import pandas as pd
import tensorflow as tf 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
sns.set(color_codes=True)
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re,string
%matplotlib inline
from sklearn import model_selection, naive_bayes, svm

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
stopwords.words('english')
import string
string.punctuation
from nltk.stem.porter import PorterStemmer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
data = pd.read_csv("/content/train.csv",header = 0)
data

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...
20963,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20964,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20965,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20966,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [ ]:
data.isna().sum()

TITLE                   0
ABSTRACT                0
Computer Science        0
Physics                 0
Mathematics             0
Statistics              0
Quantitative Biology    0
Quantitative Finance    0
dtype: int64

In [ ]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].unique())

['Reconstructing Subject-Specific Effect Maps'
 'Rotation Invariance Neural Network'
 'Spherical polyharmonics and Poisson kernels for polyharmonic functions'
 ... 'Analysing Soccer Games with Clustering and Conceptors'
 'On the Efficient Simulation of the Left-Tail of the Sum of Correlated Log-normal Variates'
 'Why optional stopping is a problem for Bayesians']
["  Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we propose a reconstruction\nmethod, n

In [ ]:
for i in range(data.shape[1]):
  print(data.iloc[:,i].value_counts())

Reconstructing Subject-Specific Effect Maps                                                                                           1
Can Deep Clinical Models Handle Real-World Domain Shifts?                                                                             1
Memory Efficient Experience Replay for Streaming Learning                                                                             1
Aspiration dynamics generate robust predictions in structured populations                                                             1
OAuthGuard: Protecting User Security and Privacy with OAuth 2.0 and OpenID Connect                                                    1
                                                                                                                                     ..
The exit time finite state projection scheme: bounding exit distributions and occupation measures of continuous-time Markov chains    1
The problem of boundary conditions for the shall

In [ ]:
data.shape

(20968, 8)

# **PRE-PROCESSING**

In [ ]:
data["Article"] = data[["TITLE", "ABSTRACT"]].apply("-".join, axis=1)
data

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Article
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,Reconstructing Subject-Specific Effect Maps- ...
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,Rotation Invariance Neural Network- Rotation ...
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,Spherical polyharmonics and Poisson kernels fo...
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,A finite element approximation for the stochas...
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,Comparative study of Discrete Wavelet Transfor...
...,...,...,...,...,...,...,...,...,...
20963,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0,Contemporary machine learning: a guide for pra...
20964,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0,Uniform diamond coatings on WC-Co hard alloy c...
20965,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0,Analysing Soccer Games with Clustering and Con...
20966,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0,On the Efficient Simulation of the Left-Tail o...


In [ ]:
def clean_text1(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

cleaned1=lambda x:clean_text1(x)

In [ ]:
data['Article']=pd.DataFrame(data.Article.apply(cleaned1))

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer

#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
from bs4 import BeautifulSoup

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [ ]:
import re,string,unicodedata

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

In [ ]:
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
data['Article']=data['Article'].apply(denoise_text)

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

In [ ]:
data['Article']=data['Article'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
data['Article']=data['Article'].apply(simple_stemmer)

In [ ]:
from nltk.corpus import stopwords


#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

{'as', 'are', 'an', 'that', 'it', 'have', 'has', 'ours', "haven't", 'or', 'aren', 'in', "doesn't", 'couldn', 'from', 'out', 'o', 'during', 'after', 'other', "hadn't", 'isn', 'his', 'mightn', 'he', 'himself', 'was', 'then', "needn't", 'd', 'under', 'any', 'against', 'those', 'some', "didn't", 'there', 'its', 'is', 'won', 'been', "hasn't", 'doesn', 'and', 'when', "you'll", 'no', 'this', "shouldn't", 'once', 'just', 'each', 'not', 'shouldn', 'their', 'doing', "you'd", 'her', 't', 'ain', 'itself', 'until', 'than', 'over', 'down', 'only', 'hasn', 'we', 'ourselves', 'between', 'didn', 'most', 'she', 'theirs', 'be', 'through', 'having', 'more', 'should', 'so', 'own', 'mustn', 'can', "you're", 'which', 'ma', 'y', 'they', 'for', 'further', 'with', 'them', "you've", "she's", 're', 'these', 'haven', "it's", 'you', 'hadn', 'what', 'below', 'both', "shan't", 'to', 'if', 'm', 'up', 'who', 'yourselves', 'into', 's', 'off', 'yours', 'do', "isn't", 'at', 'll', 'very', 'yourself', 'our', 'whom', "should

In [ ]:
data['Article']=data['Article'].apply(remove_stopwords)

In [ ]:
from nltk.stem import WordNetLemmatizer,SnowballStemmer

def simple_stemmer(text):
    ps = SnowballStemmer(language='english')
    return ' '.join([ps.stem(word) for word in tokenizer.tokenize(text)])

In [ ]:
data['Article'][0]

'reconstruct subjectspecif effect map predict model allow subjectspecif infer analyz diseas relat alter neuroimag data given subject data infer made two level global ie identifiy condit presenc subject local ie detect condit effect individu measur extract subject data global infer wide use local infer use form subjectspecif effect map rare use becaus exist model often yield noisi detect compos dispers isol island thi articl propos reconstruct method name rsm improv subjectspecif detect predict model approach particular binari classifi rsm specif aim reduc nois due sampl error associ use finit sampl exampl train classifi propos method wrappertyp algorithm use differ binari classifi diagnost manner ie without inform condit presenc reconstruct pose maximumaposteriori problem prior model whose paramet estim train data classifierspecif fashion experiment evalu perform synthet gener data data alzheim diseas neuroimag initi adni databas result synthet data demonstr use rsm yield higher detect

In [ ]:
data

,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Article
0,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0,reconstruct subjectspecif effect map predict m...
1,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0,rotat invari neural network rotat invari trans...
2,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0,spheric polyharmon poisson kernel polyharmon f...
3,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0,finit element approxim stochast maxwelllandaul...
4,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0,compar studi discret wavelet transform wavelet...
...,...,...,...,...,...,...,...,...,...
20963,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0,contemporari machin learn guid practition phys...
20964,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0,uniform diamond coat wcco hard alloy cut inser...
20965,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0,analys soccer game cluster conceptor present n...
20966,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0,effici simul lefttail sum correl lognorm varia...


In [ ]:
#word cloud for positive reviews
from wordcloud import WordCloud,STOPWORDS
positive_data = df['Article']
positive_data_string = ' '.join(positive_data)
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000, width=1200, height=600,background_color="white").generate(positive_data_string)
plt.imshow(wc , interpolation = 'bilinear')
plt.axis('off')
plt.title('Word Cloud',fontsize = 20)
plt.show()

# **TFIDF Vectors**

In [ ]:
data1=data[{'Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance','Article'}]
data1

,Physics,Article,Computer Science,Quantitative Biology,Quantitative Finance,Statistics,Mathematics
0,0,reconstruct subjectspecif effect map predict m...,1,0,0,0,0
1,0,rotat invari neural network rotat invari trans...,1,0,0,0,0
2,0,spheric polyharmon poisson kernel polyharmon f...,0,0,0,0,1
3,0,finit element approxim stochast maxwelllandaul...,0,0,0,0,1
4,0,compar studi discret wavelet transform wavelet...,1,0,0,1,0
...,...,...,...,...,...,...,...
20963,1,contemporari machin learn guid practition phys...,1,0,0,0,0
20964,1,uniform diamond coat wcco hard alloy cut inser...,0,0,0,0,0
20965,0,analys soccer game cluster conceptor present n...,1,0,0,0,0
20966,0,effici simul lefttail sum correl lognorm varia...,0,0,0,1,1


In [ ]:
print("\n\nWith TFIDFVectorizer")
vectorizer = TfidfVectorizer(max_features=10)
X = vectorizer.fit_transform(data1.Article)
print(vectorizer.get_feature_names_out())
print(X.toarray())
print("\n")
#print(cosine_similarity(X))



With TFIDFVectorizer
['data' 'gener' 'method' 'model' 'network' 'result' 'show' 'system' 'thi'
 'use']
[[0.72425956 0.08379927 0.17205836 ... 0.         0.05137542 0.5105419 ]
 [0.         0.         0.         ... 0.         0.31188669 0.19371038]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.17202716 0.         0.         ... 0.         0.         0.60632318]
 [0.         0.         0.44003749 ... 0.47959423 0.         0.        ]
 [0.         0.         0.72415862 ... 0.         0.43245733 0.53719173]]




In [ ]:
print("\n\nWith TFIDFVectorizer and removing stop words")
vectorizer = TfidfVectorizer(stop_words=nlp.Defaults.stop_words,max_features=10)
X = vectorizer.fit_transform(data1.Article)
print(vectorizer.get_feature_names_out())
print(X.toarray())
print("\n")
#print(cosine_similarity(X))



With TFIDFVectorizer and removing stop words


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


['data' 'gener' 'method' 'model' 'network' 'problem' 'result' 'system'
 'thi' 'use']
[[0.72976199 0.08443592 0.17336555 ... 0.         0.05176573 0.51442065]
 [0.         0.         0.         ... 0.         0.31188669 0.19371038]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.17202716 0.         0.         ... 0.         0.         0.60632318]
 [0.         0.         0.39862582 ... 0.43445989 0.         0.        ]
 [0.         0.         0.39460872 ... 0.         0.23565477 0.29272667]]




## **LOGISTIC REGRESSION**

In [ ]:
X= data1['Article']
y=data1.drop("Article", axis=1)
X.head

<bound method NDFrame.head of 0        reconstruct subjectspecif effect map predict m...
1        rotat invari neural network rotat invari trans...
2        spheric polyharmon poisson kernel polyharmon f...
3        finit element approxim stochast maxwelllandaul...
4        compar studi discret wavelet transform wavelet...
                               ...                        
20963    contemporari machin learn guid practition phys...
20964    uniform diamond coat wcco hard alloy cut inser...
20965    analys soccer game cluster conceptor present n...
20966    effici simul lefttail sum correl lognorm varia...
20967    whi option stop problem bayesian recent option...
Name: Article, Length: 20968, dtype: object>

In [ ]:
y

,Physics,Computer Science,Quantitative Biology,Quantitative Finance,Statistics,Mathematics
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,1,0,0,1,0
...,...,...,...,...,...,...
20963,1,1,0,0,0,0
20964,1,0,0,0,0,0
20965,0,1,0,0,0,0
20966,0,0,0,0,1,1


In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4, random_state=0)

In [ ]:
# Convert text to vectors using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
import sklearn
lg = sklearn.linear_model.LogisticRegression()

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
model =  MultiOutputClassifier(lg)

In [ ]:
model.fit(X_train,y_train)

MultiOutputClassifier(estimator=LogisticRegression())

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, model.predict(X_test))*100))

Accuracy on test data: 63.4%


# **WORD2VEC**


In [ ]:
from gensim.models import Word2Vec

In [ ]:
sentences = [review.split() for review in data['Article']]
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)
model.save('word2vec.model')

In [ ]:
# Convert each word to vector and represent the sentence in vector form using the word embeddings
def sentence_vector(sentence,model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.vocab]
    if len(word_vectors) == 0:
        return np.zeros((100,))
    return np.mean(word_vectors, axis=0)


##LOGISTIC REGRESSION



In [ ]:
# Convert each sentence in the dataset to a vector using the Word2Vec model
word2vec_train = np.array([sentence_vector(sentence, model) for sentence in data1['Article']])
word2vec_test = np.array(y)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(word2vec_train, word2vec_test, test_size=0.20, random_state=0)

In [ ]:
lg = sklearn.linear_model.LogisticRegression()

In [ ]:
model1 =  MultiOutputClassifier(lg)

In [ ]:
model1.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

MultiOutputClassifier(estimator=LogisticRegression())

In [ ]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, model1.predict(X_test))*100))

Accuracy on test data: 63.5%


# **COMBINING TFIDF AND WORD2VEC**

In [ ]:
def combine_vectors(doc):
    tfidf_vec = vectorizer.transform([doc])
    w2v_vec = sentence_vector(doc, model)
    combined_vec = np.concatenate([np.squeeze(tfidf_vec.toarray()), w2v_vec])
    return combined_vec

##LOGISTIC REGRESSION

In [ ]:
combined_train = np.array([combine_vectors(doc) for doc in data['Article']])
combined_test = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(combined_train, combined_test, test_size=0.25, random_state=0)

In [ ]:
lg2 = sklearn.linear_model.LogisticRegression()

In [ ]:
model2 =  MultiOutputClassifier(lg2)

In [ ]:
model2.fit(X_train,y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

MultiOutputClassifier(estimator=LogisticRegression())

In [ ]:
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, model2.predict(X_test))*100))

Accuracy on test data: 65.5%
